# Flood  - Classification Using Logistic Regression

<h3> 
Aaron Trefler <br/>
JPL <br/>
Created: 07/11/2016 <br/>
</h3>

# I. Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.io as sio

from flood_functions import grace_brick_convert_lowres 
from flood_functions import calculate_confusion_matrix_bricks
from flood_functions import calculate_ml_metric_maps
from IPython.display import display, HTML
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [3]:
%matplotlib

Using matplotlib backend: MacOSX


In [4]:
# define directories
dir_flood = '../../Raw Data/Flood Observatory/'
dir_python_data = '../Data/'
dir_grace = '../../Work_Matlab/Data/'
dir_figures = '../Figures/'

# II. Load Data

## GRACE

### Features

In [5]:
# GRACE MASCON-GRI Pickle Files
f = open(dir_python_data+'grace_features.p', 'rb')
grace_features_dict = pickle.load(f)
f.close()

In [6]:
# assign grace maps to python objects

# continuous
grace_lwe = grace_features_dict['grace_lwe']
grace_lwe_norm = grace_features_dict['grace_lwe_norm']
grace_lwe_clim_norm = grace_features_dict['grace_lwe_clim_norm']
grace_lwe_noClim_norm = grace_features_dict['grace_lwe_noClim_norm']

# ranked
grace_lwe_rank = grace_features_dict['grace_lwe_rank']
grace_lwe_rank_norm = grace_features_dict['grace_lwe_rank_norm']
grace_lwe_clim_rank = grace_features_dict['grace_lwe_clim_rank']
grace_lwe_clim_rank_norm = grace_features_dict['grace_lwe_clim_rank_norm']
grace_lwe_noClim_rank = grace_features_dict['grace_lwe_noClim_rank']
grace_lwe_noClim_rank_norm = grace_features_dict['grace_lwe_noClim_rank_norm']

## Flood Observatory

### Flood Event Brick

In [7]:
f = open(dir_python_data+'flood_event_brick.p', 'rb')
flood_event_brick = pickle.load(f)
f.close()
flood_event_brick_highres = flood_event_brick[0]

### Dataframe

In [8]:
# name of preprocessed and merged flood dataframe
df_name = 'df_flood_grace_time_location_features'

In [9]:
# FO data
df_flood_grace = pd.read_csv(dir_python_data + df_name + '.csv')
df_flood_grace = df_flood_grace.drop('Unnamed: 0', axis=1)
#df_flood_grace.tail(1)

## Script Variables

In [10]:
# GRACE
dim = grace_lwe.shape
d1 = dim[0]
d2 = dim[1]
tp = dim[2]
# FO
floods = len(df_flood_grace)

# III. Classification

### Setup

In [11]:
# define feature brick (NaN's for non-land regions)
feature_brick_highres = grace_lwe_rank

In [12]:
# create low resolution feature brick (0's and 1's)
feature_brick = grace_brick_convert_lowres(feature_brick_highres, 6)

In [13]:
# create low resolution flood brick (0's and 1's)
flood_event_brick_lowres = grace_brick_convert_lowres(flood_event_brick_highres, 6)
flood_event_brick_lowres[flood_event_brick_lowres > 0] = 1

In [14]:
# create binary flood zone map and brick
flood_zone = np.nansum(flood_event_brick_lowres,axis=2)
flood_zone[flood_zone > 0] = 1
flood_zone = flood_zone[:,:,np.newaxis]
flood_zone_brick = np.tile(flood_zone,(1,1,152))

In [15]:
# create label brick (NaNs for non-flood regions)
label_brick = flood_event_brick_lowres
label_brick[flood_zone_brick==0] = np.nan

In [26]:
# define low resolution variabls
dim_lowres = label_brick.shape
d1_lowres = dim_lowres[0]  
d2_lowres = dim_lowres[1]

### Logistic Regression

In [52]:
# create prediction brick
pred_brick = np.empty([d1_lowres,d2_lowres,tp])
pred_brick.fill(np.nan)

# perform logistic regression
for i in range(d1_lowres):
    
    for j in range(d2_lowres):
        
        features = feature_brick[i,j,:]
        labels = label_brick[i,j,:].astype(int)
        
        # add column of 1's to features
        ones = np.ones(len(features))
        features = np.vstack((ones,features)).transpose()
              
        if (np.nansum(labels) > 0): #flood events occured
                
            clf = LogisticRegression(fit_intercept=False)
            clf.fit(features, labels)
            pred = clf.predict(features)
            pred_brick[i,j,:] = pred    

### Threshold Classification

In [78]:
# create prediction brick
pred_brick = np.empty([d1_lowres,d2_lowres,tp])
pred_brick.fill(np.nan)

# perform threshold classification
for i in range(d1_lowres):
    
    for j in range(d2_lowres):
        
        features = feature_brick[i,j,:]
        labels = label_brick[i,j,:].astype(int)
              
        if (np.nansum(labels) > 0): #flood events occured
                
            pred = features >= 114
            pred_brick[i,j,:] = pred    

### Null Hypothesis Prediction

In [79]:
# create null hypothesis prediction brick
pred_null_brick = np.empty([d1_lowres,d2_lowres,tp])
pred_null_brick.fill(np.nan)

# fill in null hypothesis birck with all 0's in areas with flood events
for i in range(d1_lowres):
    
    for j in range(d2_lowres):
        
        labels = label_brick[i,j,:].astype(int)
              
        if (np.nansum(labels) > 0): #flood events occured
                
            pred_null_brick[i,j,:] = 0  

### Calculate Metrics

In [80]:
# create confusion-matrix bricks

# predcitions
pred_confusion_matrix = calculate_confusion_matrix_bricks(label_brick, pred_brick)
pred_tp_brick = pred_confusion_matrix['true_positive'] #hit
pred_fp_brick = pred_confusion_matrix['false_positive'] #type I error
pred_fn_brick = pred_confusion_matrix['false_negative'] #type II error
pred_tn_brick = pred_confusion_matrix['true_negative'] #correct regection

# null hypothesis
pred_null_confusion_matrix = calculate_confusion_matrix_bricks(label_brick, pred_null_brick)
pred_null_tp_brick = pred_null_confusion_matrix['true_positive'] #hit
pred_null_fp_brick = pred_null_confusion_matrix['false_positive'] #type I error
pred_null_fn_brick = pred_null_confusion_matrix['false_negative'] #type II error
pred_null_tn_brick = pred_null_confusion_matrix['true_negative'] #correct regection

In [89]:
# create ML metric maps

# predictions
pred_ml_metrics = calculate_ml_metric_maps(label_brick, \
                                           pred_brick, \
                                           pred_tp_brick, \
                                           pred_fp_brick, \
                                           pred_tn_brick, \
                                           pred_fn_brick)
pred_accuracy_map = pred_ml_metrics['accuracy']
pred_missClassRate_map = pred_ml_metrics['missClassRate']
pred_tpRate_map = pred_ml_metrics['tpRate']
pred_fpRate_map = pred_ml_metrics['fpRate']
pred_specificity_map = pred_ml_metrics['specificity']
pred_precision_map = pred_ml_metrics['precision']
pred_prevalence_map = pred_ml_metrics['prevalence']

# null hypothesis
pred_null_ml_metrics = calculate_ml_metric_maps(label_brick, \
                                           pred_null_brick, \
                                           pred_null_tp_brick, \
                                           pred_null_fp_brick, \
                                           pred_null_tn_brick, \
                                           pred_null_fn_brick)
pred_null_accuracy_map = pred_null_ml_metrics['accuracy']
pred_null_missClassRate_map = pred_null_ml_metrics['missClassRate']
pred_null_tpRate_map = pred_null_ml_metrics['tpRate']
pred_null_fpRate_map = pred_null_ml_metrics['fpRate']
pred_null_specificity_map = pred_null_ml_metrics['specificity']
pred_null_precision_map = pred_null_ml_metrics['precision']
pred_null_prevalence_map = pred_null_ml_metrics['prevalence']

# NULL

In [85]:
# create ML metric maps

#accuracy
metric_accuracy_map = np.empty([d1_lowres,d2_lowres]) 
metric_accuracy_map.fill(np.nan)
#miss-classifcation rate
metric_missClassRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_missClassRate_map.fill(np.nan)
#true positive rate
metric_tpRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_tpRate_map.fill(np.nan)
#false positive rate
metric_fpRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_fpRate_map.fill(np.nan)
#specificity
metric_specificity_map = np.empty([d1_lowres,d2_lowres]) 
metric_specificity_map.fill(np.nan)
#precision
metric_precision_map = np.empty([d1_lowres,d2_lowres]) 
metric_precision_map.fill(np.nan)
#prevalence
metric_prevalence_map = np.empty([d1_lowres,d2_lowres]) 
metric_prevalence_map.fill(np.nan)

for i in range(d1_lowres):
    for j in range(d2_lowres):
    
        if (np.nansum(label_brick[i,j,:]) > 0): #flood events occured
            
            metric_accuracy_map[i,j] = \
                (np.sum(pred_null_tp_brick[i,j,:]) + np.sum(pred_null_tn_brick[i,j,:])) / tp
            metric_missClassRate_map[i,j] = \
                (np.sum(pred_null_fp_brick[i,j,:]) + np.sum(pred_null_fn_brick[i,j,:])) / tp
            metric_tpRate_map[i,j] = \
                np.sum(pred_null_tp_brick[i,j,:]) /  np.sum(label_brick[i,j,:])
            metric_fpRate_map[i,j] = \
                np.sum(pred_null_fp_brick[i,j,:]) /  np.sum(label_brick[i,j,:] == 0)
            metric_specificity_map[i,j] = \
                np.sum(pred_null_tn_brick[i,j,:]) /  np.sum(label_brick[i,j,:] == 0)
            metric_precision_map[i,j] = \
                np.sum(pred_null_tp_brick[i,j,:]) /  np.sum(pred_null_brick[i,j,:] == 1)
            metric_prevalence_map[i,j] = \
                np.sum(label_brick[i,j,:] == 1)  / tp

In [86]:
plt.subplot(3,3,1)
plt.imshow(np.flipud(metric_accuracy_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Accuracy')

plt.subplot(3,3,2)
plt.imshow(np.flipud(metric_missClassRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Error Rate')

plt.subplot(3,3,3)
plt.imshow(np.flipud(metric_tpRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('True Positive Rate')

plt.subplot(3,3,4)
plt.imshow(np.flipud(metric_fpRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('False Positive Rate')

plt.subplot(3,3,5)
plt.imshow(np.flipud(metric_specificity_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Specificity')

plt.subplot(3,3,6)
plt.imshow(np.flipud(metric_precision_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Precision')

plt.subplot(3,3,7)
plt.imshow(np.flipud(metric_prevalence_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Prevalence')

plt.show()

# PREDICTION

In [87]:
# create ML metric maps

#accuracy
metric_accuracy_map = np.empty([d1_lowres,d2_lowres]) 
metric_accuracy_map.fill(np.nan)
#miss-classifcation rate
metric_missClassRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_missClassRate_map.fill(np.nan)
#true positive rate
metric_tpRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_tpRate_map.fill(np.nan)
#false positive rate
metric_fpRate_map = np.empty([d1_lowres,d2_lowres]) 
metric_fpRate_map.fill(np.nan)
#specificity
metric_specificity_map = np.empty([d1_lowres,d2_lowres]) 
metric_specificity_map.fill(np.nan)
#precision
metric_precision_map = np.empty([d1_lowres,d2_lowres]) 
metric_precision_map.fill(np.nan)
#prevalence
metric_prevalence_map = np.empty([d1_lowres,d2_lowres]) 
metric_prevalence_map.fill(np.nan)

for i in range(d1_lowres):
    for j in range(d2_lowres):
    
        if (np.nansum(label_brick[i,j,:]) > 0): #flood events occured
            
            metric_accuracy_map[i,j] = \
                (np.sum(pred_tp_brick[i,j,:]) + np.sum(pred_tn_brick[i,j,:])) / tp
            metric_missClassRate_map[i,j] = \
                (np.sum(pred_fp_brick[i,j,:]) + np.sum(pred_fn_brick[i,j,:])) / tp
            metric_tpRate_map[i,j] = \
                np.sum(pred_tp_brick[i,j,:]) /  np.sum(label_brick[i,j,:])
            metric_fpRate_map[i,j] = \
                np.sum(pred_fp_brick[i,j,:]) /  np.sum(label_brick[i,j,:] == 0)
            metric_specificity_map[i,j] = \
                np.sum(pred_tn_brick[i,j,:]) /  np.sum(label_brick[i,j,:] == 0)
            metric_precision_map[i,j] = \
                np.sum(pred_tp_brick[i,j,:]) /  np.sum(pred_brick[i,j,:] == 1)
            metric_prevalence_map[i,j] = \
                np.sum(label_brick[i,j,:] == 1)  / tp

In [88]:
plt.figure()

plt.subplot(3,3,1)
plt.imshow(np.flipud(metric_accuracy_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Accuracy')

plt.subplot(3,3,2)
plt.imshow(np.flipud(metric_missClassRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Error Rate')

plt.subplot(3,3,3)
plt.imshow(np.flipud(metric_tpRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('True Positive Rate')

plt.subplot(3,3,4)
plt.imshow(np.flipud(metric_fpRate_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('False Positive Rate')

plt.subplot(3,3,5)
plt.imshow(np.flipud(metric_specificity_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Specificity')

plt.subplot(3,3,6)
plt.imshow(np.flipud(metric_precision_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Precision')

plt.subplot(3,3,7)
plt.imshow(np.flipud(metric_prevalence_map.transpose()), vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')
plt.title('Prevalence')

plt.show()